In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import feature, exposure
from matplotlib.pyplot import bar
from skimage.transform import resize
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from PIL import Image, ImageEnhance
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
import os

In [4]:
from skimage import feature, exposure

def HistogramofOrientedGradients(image):
    resized_img = resize(image, (128, 64))
    fd, hog_image = feature.hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
    
    return fd

In [5]:
skew_detected_female = []
for dirname, _, filenames in os.walk('../input/cmp23-handwritten-males-vs-females/Females/Females/'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename),0)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            skew_detected_female.append(HistogramofOrientedGradients(img))
skew_detected_female = np.array(skew_detected_female)

skew_detected_male = []
for dirname, _, filenames in os.walk('../input/cmp23-handwritten-males-vs-females/Males/Males/'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename),0)
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None:
            skew_detected_male.append(HistogramofOrientedGradients(img))
skew_detected_male = np.array(skew_detected_male)

In [7]:
# classifier
X=np.concatenate((skew_detected_male,skew_detected_female),axis=0)

Y=np.concatenate((np.ones(skew_detected_male.shape[0]),
                np.zeros(skew_detected_female.shape[0])),
                axis=0)

In [8]:
X,Y=sklearn.utils.shuffle(X,Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% test

In [9]:
svm = LinearSVC()
clf = CalibratedClassifierCV(svm) 
clf.fit(X_train, y_train)

In [10]:
y_pred = clf.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [11]:
clf.probability=False
prob = clf.predict_proba(X_test)
print(prob)
#print(y_pred)
#print(y_test)

In [12]:
model = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', criterion = 'gini', n_jobs=-1)

# Train the model using the training sets
model.fit(X_train, y_train)

In [13]:
y_pred = model.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [14]:
model2 = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [15]:
model4 = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(2,), 
                    random_state=1)

model4.fit(X_train, y_train) 
y_pred = model4.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))